# Rhme2 hard SCA challenge in Jlsca

Requires:

* The 305 MB traceset tarball from https://drive.google.com/open?id=0B2slHLSL3nXaOHdhTXpMMHZkNjQ. Uncompress it next to this notebook.
* Jlsca, PyCall and PyPlot packages
* IJulia package

In [ ]:
# load the packages

using Jlsca.Sca
using Jlsca.Trs
using Jlsca.Align
using PyCall
using PyPlot.plot,PyPlot.figure

@pyimport numpy

## Convert traces from Chipwhisperer capture to Trs format

In [ ]:
# prefix identifying the capture, bulky as it is
# (so far have been lazy to automate based on ChipWhisperer config file)
prefix = "hardsca_longer_traces/2017.01.27-17.45.58_"

samples = numpy.load("$(prefix)traces.npy");
input = numpy.load("$(prefix)textin.npy");
output = numpy.load("$(prefix)textout.npy");

numberOfTraces = size(input)[1] # because it happens that CW software saves more traces then inputs o_O
numberOfSamples = size(samples)[2]
dataSpace = size(input)[2] + size(output)[2]
sampleType = Float32;

In [ ]:
trs = InspectorTrace("hardsca_longer_traces.trs", dataSpace, sampleType, numberOfSamples)
for t in 1:numberOfTraces
  trs[t] = (vcat(input[t,:],output[t,:]), map(Float32, samples[t,:]))
end
close(trs)

## Explore traces

In [ ]:
trs = InspectorTrace("hardsca_longer_traces.trs")

In [ ]:
((data,samples),eof) = readTraces(trs, 1:100)

Plot 3 traces

In [ ]:
for i in 1:3; figure(); plot(samples[i,:]); end

Zoom in at the end, plot a few, observe the alignment is not very good

In [ ]:
zoomstart = 18000
zoomend = 22000

for i in 1:4; figure(); plot(samples[i,zoomstart:zoomend]); end

Try some alignment, and see the result

In [ ]:
reset(trs)

maxShift = 20000
referencestart = zoomstart + 1000
referenceend = referencestart + 1500
reference = trs[3][2][referencestart:referenceend]
corvalMin = 0.0
alignstate = CorrelationAlignFFT(reference, referencestart, maxShift)
addSamplePass(trs, x -> ((shift,corval) = correlationAlign(x, alignstate); corval > corvalMin ? circshift(x, shift) : Vector{eltype(x)}(0)))

((data,samples),eof) = readTraces(trs, 1:4)

popSamplePass(trs)

for i in 1:4; figure(); plot(samples[i,18000:22000]); end

In [ ]:
close(trs)

## CPA attack with alignment

In [ ]:
trs = InspectorTrace("hardsca_longer_traces.trs")

params = AesSboxAttack()
params.dataOffset = 17
params.direction = BACKWARD

numberOfAverages = length(params.keyByteOffsets)
numberOfCandidates = getNumberOfCandidates(params)

numberOfTraces = length(trs)


Prepare the alignment pass, bump the corvalMin to throw away stuff that is poorly aligned

We're also throwing away everything outside our window of interest before it goes into the attack, after alignment

In [ ]:
popSamplePass(trs)
popSamplePass(trs)

zoomstart = 18000
zoomend = 22000
maxShift = 20000
referencestart = zoomstart + 1000
referenceend = referencestart + 1500
reference = trs[3][2][referencestart:referenceend]
corvalMin = 0.4
alignstate = CorrelationAlignFFT(reference, referencestart, maxShift)

addSamplePass(trs, x -> ((shift,corval) = correlationAlign(x, alignstate); corval > corvalMin ? circshift(x, shift) : Vector{eltype(x)}(0)))
addSamplePass(trs, x -> x[zoomstart-1000:zoomend])

Run with conditional averaging

In [ ]:
reset(trs) # note, this resets the conditional averager, it doesn't pop the sample passes

params.analysis = DPA()
params.analysis.statistic = cor
#params.analysis.leakageFunctions = [x -> ((x .>> i)&1) for i in 0:7]
params.analysis.leakageFunctions = [hw]

setPostProcessor(trs, CondAvg(SplitByData(numberOfAverages,numberOfCandidates)))  
sca(trs, params, 1, numberOfTraces)
